In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.metrics import accuracy_score, roc_auc_score,mean_squared_error
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, ARDRegression
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor

In [2]:
X = np.genfromtxt("X_train.txt", delimiter=None)
Y = np.genfromtxt("Y_train.txt", delimiter=None)
Xtest = np.genfromtxt("X_test.txt",delimiter=None)
X,Y = ml.shuffleData(X,Y)
Xtr, Xte, Ytr, Yte = ml.splitData(X, Y, 0.75)

In [ ]:
X = np.genfromtxt("X_train.txt", delimiter=None)[:10000] 
Y = np.genfromtxt("Y_train.txt", delimiter=None)[:10000] 
Xtest = np.genfromtxt("X_test.txt",delimiter=None)
X,Y = ml.shuffleData(X,Y)
Xtr, Xte, Ytr, Yte = ml.splitData(X, Y, 0.75)

K = [1,2,5,10,50,100,200,500,1000,1500,2000,5000,7500]
knnpredict = []
knnpredict1 = []
knnVMse = []
knnTMse = []
for i,k in enumerate(K):
    knn = KNeighborsRegressor(n_neighbors = k)
    knn.fit(Xtr[:, 0:4], Ytr)
    knnpredict.append(knn.predict(Xte[:, 0:4]))
    knnpredict1.append(knn.predict(Xtr[:, 0:4]))

for p in knnpredict:
    knnVMse.append(mean_squared_error(Yte, p))
    
for p in knnpredict1:
    knnTMse.append(mean_squared_error(Ytr,p))
    
for i in range(len(knnVMse)):
    print "k = {:04d}: VMse = {:05f}, TMse = {:05f}".format(K[i],knnVMse[i],knnTMse[i])

In [ ]:
bestK = K[knnVMse.index(min(knnVMse))]
print bestK

print len(knnpredict)

In [13]:
clfs = [KMeans(n_clusters=8, init='random'),
        KNeighborsRegressor(n_neighbors = 200),
        GradientBoostingRegressor(n_estimators=16000)
       ]

temp_results_t = np.zeros((  len(Ytr)        ,len(clfs) ))   #x1,x2
temp_results_v = np.zeros((  len(Yte)        ,len(clfs) ))   #y1,y2
test1        = np.zeros((  Xtest.shape[0] ,len(clfs) ))   
for i, clf in enumerate(clfs):
    clf.fit(Xtr,Ytr)
    temp_results_v[:,i] = clf.predict(Xte)
    temp_results_t[:,i] = clf.predict(Xtr)
    test1[:,i] = clf.predict(Xtest)

lr = LinearRegression(normalize=True)
lr.fit(temp_results_v, Yte)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [14]:
print "AUC on Train data: ",roc_auc_score(Ytr, lr.predict(temp_results_t))
print "AUC on Valid data: ",roc_auc_score(Yte, lr.predict(temp_results_v))

AUC on Train data:  0.936152812476
AUC on Valid data:  0.756755744071


In [16]:
Ypred_lr = lr.predict(test1)
np.savetxt('Yhat_knn_lr_3_15.txt', np.vstack( (np.arange(len(Ypred_lr)) , Ypred_lr) ).T, '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');